In [1]:
import os
from exchangelib import Account, Credentials, Mailbox, Message, Configuration
import logging


logger = logging.getLogger(__name__)

# email account which will send the feedback emails
EXCHANGE_ACCOUNT = os.getenv("EXCHANGE_ACCOUNT")
EXCHANGE_PW = os.getenv("EXCHANGE_PW")
EXCHANGE_EMAIL = os.getenv("EXCHANGE_EMAIL")
# email addresses to which the feedback emails will be sent
RECIPIENTS = os.getenv("RECIPIENTS").split(",")



def send_feedback_email(subject, body):
    try:
        credentials = Credentials(EXCHANGE_ACCOUNT, EXCHANGE_PW)
        account = Account(
            EXCHANGE_EMAIL, credentials=credentials, autodiscover=True
        )
        recipients = [
            Mailbox(email_address=recipient) for recipient in RECIPIENTS
        ]
        mail = Message(
            account=account,
            folder=account.sent,
            subject=subject,
            body=body,
            to_recipients=recipients,
        )
        mail.send_and_save()
    except Exception as ex:
        logger.warning(f"Couldn't send feedback email. Exception raised: {ex}.")


In [2]:
EXCHANGE_EMAIL

'service@rl-institut.de'

In [3]:
date = "2021-09-21"
SUBJECT = "test"
FEEDBACK = "text about test"
Name = "pf"
email = "test@rl-institut.de"
username = "testUser" 

body = f"""
Feedback form for Energy Planning Application

Received Feedback
-----------------

Date: {date}
Topic: {SUBJECT}
Content: {FEEDBACK}

Infromation about sender
------------------------
Name: {Name}
E-mail Adress: {email}
User-ID: {username}

"""

send_feedback_email("[E-LAND] [EPA] Feedback for EPA submitted by {Name}", body)

In [1]:
from exchangelib import Account, Credentials, Mailbox, Message


EXCHANGE_ACCOUNT = "service@rl-institut.de"
EXCHANGE_PW = "dQ2CEwV0K4Z"
EXCHANGE_EMAIL = "service@rl-institut.de"

RECIPIENTS = ["pierre-francois.duc@rl-institut.de", "martha.hoffmann@rl-institut.de"]

credentials = Credentials(EXCHANGE_ACCOUNT, EXCHANGE_PW)
account = Account(
    EXCHANGE_EMAIL, credentials=credentials, autodiscover=True
)
recipients = [
    Mailbox(email_address=recipient) for recipient in RECIPIENTS
]
mail = Message(
    account=account,
    folder=account.sent,
    subject="Betreff",
    body="Das ist eine Nachricht",
    to_recipients=recipients,
)
mail.send_and_save()

In [29]:
# CREATE deepL translation

german_file = "german_to_mix.txt"
po_in_file = "to_translate.txt"
po_out_file = "translation_to_proof.txt"

# deepL translation of django.po
with open(f"locale/de/LC_MESSAGES/{german_file}","r") as fp:
    lines = fp.readlines()

msgids = []
msgstr = []
long_line = False

for l in lines:
    #print("line: " + l)
    #print("line short: " + l[:6])
    
    if l[:5] == "msgid":
        if l[6:8] == '""':
            long_line = True
        else:
            msgids.append(l.replace("msgid", "msgstr"))
    elif l[:6] == "msgstr":
        #print(msgstr)
        long_line = False
        if len(msgstr) > 0:
            msgids.append(tuple(msgstr))
            msgstr = []
    elif long_line is True:
        msgstr.append(l)
    #print(long_line)

with open(f"locale/de/LC_MESSAGES/{po_in_file}","r") as fp:
    tr_lines = fp.readlines()

i = -1
new_lines = []
for l in tr_lines:
    if l[:6] == "msgstr":
        if i ==-1:
            new_lines.append(l)
        elif i > -1:
            translation = msgids[i]
            if isinstance(translation, str):
                new_lines.append(translation)
            else:
                new_lines.append('msgstr "" \n')
                for tr in translation:
                    new_lines.append(tr + "\n")
        i = i + 1
    else:
        new_lines.append(l)

# deepL translation inserted into django.po
with open(f"locale/de/LC_MESSAGES/{po_out_file}","w") as fp:
    fp.writelines(new_lines)

In [27]:
import pandas as pd

df = pd.read_csv("cdn_static_root/MVS_parameters_list.csv", index_col="label")
#df = df.set_index("label")

In [2]:
import csv

with open("cdn_static_root/MVS_parameters_list.csv") as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='"')
    parameter_dict = {}
    for i, row in enumerate(spamreader):
        #print(row)
        if i == 0:
            hdr = row
            label_idx = hdr.index("label")
            verbose_idx = hdr.index("verbose")
        else:
            label = row[label_idx]
            verbose = row[verbose_idx]
            if verbose != "None":
                row[label_idx] = verbose
            parameter_dict[label] = {k: v for k, v in zip(hdr,row)}
            
        # get the label out and map the label to a dict formed with headers and each row



In [3]:
verbose_idx
parameter_dict["discount_factor"]

{'verbose': 'Discount factor',
 ':Default:': 'None',
 ':Definition:': 'Discount factor is the factor which accounts for the depreciation in the value of money in the future, compared to the current value of the same money. The common method is to calculate the weighted average cost of capital (WACC) and use it as the discount rate.',
 ':Example:': '0.06',
 ':Restrictions:': 'Between 0 and 1',
 ':Type:': 'numeric',
 ':Unit:': 'Factor',
 'label': 'Discount factor',
 'ref': 'discountfactor-label',
 'category': 'economic_data',
 'see_also': ''}

In [24]:
# procedure:
# 1. make sure all descriptions and units from the parameter description file are added to the pickle (add some lines in forms.py)
# 2. check that there is no duplicate of existing translations in the po file (TODO write a script to compare the msgid)
# 3. write a file ("to_translate.txt") to translate with the script of this page
# 4. translate it on deepL
# 5. paste the content of deepL in the file "german_to_mix.txt"
# 6. Add an empty msgid, msgstr couple at the beginning of the "to_translate.txt" file 
# 7. Run the script "CREATE deepL translation"
# 8. send the file "translation_to_proof.txt" to proof the translation
# 9. copy the content of the file in the django.po file (except the empty msgid, msgstr couple)
# 10. run python manage.py compilemessages
# 11. DONE :)



import pickle
import os

lang = "de"

with open(f"cdn_static_root/personal_translation_{lang}.pickle", 'rb') as handle:
    trans_dict = pickle.load(handle)

with open(os.path.join("locale", lang, "LC_MESSAGES", f'personal_translation_{lang}.po'),'w') as fp:
    lines = []
    for item in trans_dict:
        item = item.replace('"', "'")
        lines = lines + [f'msgid "{item}"', 'msgstr ""', ""]
    fp.writelines("\n".join(lines))


In [40]:
b

{'A fixed cost to implement the asset, eg. planning costs which do not depend on the (optimized) asset capacity.': '',
 'Actual CAPEX of an asset, i.e., specific investment costs': '',
 'Actual OPEX of an asset, i.e., specific operational and maintenance costs.': '',
 'Variable cost associated with a flow through/from the asset (eg. Euro/kWh).': '',
 'Number of operational years of the asset until it has to be replaced.': '',
 'Ratio of energy output/energy input. The battery efficiency is the ratio of the energy taken out from the battery, to the energy put into the battery. It means that it is not possible to retrieve as much energy as provided to the battery due to the discharge losses. The efficiency of the "input power" and "ouput power" columns should be set equal to the charge and dischage efficiencies respectively, while the "storage capacity" efficiency should be equal to the storage\'s efficiency/ability to hold charge over time (`= 1 - self-discharge/decay`), which is usuall